# Camera Event Producer A

This notebook implements a Kafka producer that reads data from `camera_event_A.csv` and sends it to a Kafka topic in batches.

In [1]:
# Import required libraries
import pandas as pd
import json
import time
from kafka import KafkaProducer
from datetime import datetime
import os

In [2]:
# Configuration parameters
KAFKA_BOOTSTRAP_SERVERS = '172.25.240.1:9092'
KAFKA_TOPIC = 'camera_events'
BATCH_INTERVAL = 5  # seconds between each batch
PRODUCER_ID = 'Producer-A'
CSV_FILE = 'camera_event_A.csv'  # Path to the CSV file

# Check if the file exists, otherwise use a default path for testing
if not os.path.exists(CSV_FILE):
    CSV_FILE = '/home/student/Assignment 2/temp_data/data/camera_event_A.csv'  # Adjust path as needed for testing
    if not os.path.exists(CSV_FILE):
        print(f"Warning: {CSV_FILE} not found. Please ensure the file exists.")

In [3]:
# Function to load data from CSV file
def load_data(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded {len(df)} records from {file_path}")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

In [4]:
# Function to initialize Kafka producer
def create_kafka_producer():
    try:
        producer = KafkaProducer(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            key_serializer=lambda k: k.encode('utf-8') if k else None
        )
        print("Kafka producer initialized successfully")
        return producer
    except Exception as e:
        print(f"Error creating Kafka producer: {e}")
        return None

In [5]:
# Function to process and send data in batches
def process_and_send_batches(producer, df):
    # Get unique batch IDs and sort them
    batch_ids = sorted(df['batch_id'].unique())
    
    for batch_id in batch_ids:
        # Get records for current batch
        batch_df = df[df['batch_id'] == batch_id].copy()
        
        # Add producer information
        batch_df['producer_info'] = PRODUCER_ID
        
        # Add current processing timestamp
        batch_df['processing_timestamp'] = datetime.now().isoformat()
        
        # Extract event date from timestamp
        batch_df['event_date'] = pd.to_datetime(batch_df['timestamp']).dt.date.astype(str)
        
        # Convert to list of dictionaries for sending
        batch_records = batch_df.to_dict('records')
        
        # Send each record in the batch to Kafka
        sent_count = 0
        for record in batch_records:
            try:
                # Use car_plate as the key for partitioning
                key = record['car_plate']
                # Send to Kafka topic
                producer.send(KAFKA_TOPIC, key=key, value=record)
                sent_count += 1
            except Exception as e:
                print(f"Error sending record: {e}")
        
        # Flush to ensure all messages are sent before waiting
        producer.flush()
        
        print(f"Sent batch {batch_id} with {sent_count}/{len(batch_records)} records from {PRODUCER_ID}")
        
        # Wait for the specified interval before sending the next batch
        time.sleep(BATCH_INTERVAL)

In [6]:
# Main execution function
def run_producer():
    # Load data from CSV
    df = load_data(CSV_FILE)
    if df is None:
        print("Failed to load data. Exiting.")
        return
    
    # Create Kafka producer
    producer = create_kafka_producer()
    if producer is None:
        print("Failed to create Kafka producer. Exiting.")
        return
    
    try:
        # Process and send data in batches
        print(f"Starting to send data from {PRODUCER_ID} at {datetime.now()}")
        process_and_send_batches(producer, df)
        print(f"Finished sending all data from {PRODUCER_ID} at {datetime.now()}")
    except KeyboardInterrupt:
        print("Producer interrupted by user")
    except Exception as e:
        print(f"Error in producer: {e}")
    finally:
        # Close producer
        if producer is not None:
            producer.close()
            print("Kafka producer closed")

In [7]:
# Run the producer
if __name__ == "__main__":
    run_producer()

Successfully loaded 556340 records from /home/student/Assignment 2/temp_data/data/camera_event_A.csv
Kafka producer initialized successfully
Starting to send data from Producer-A at 2025-05-29 10:07:53.968425
Sent batch 1 with 20/20 records from Producer-A
Sent batch 2 with 20/20 records from Producer-A
Sent batch 3 with 20/20 records from Producer-A
Sent batch 4 with 20/20 records from Producer-A
Sent batch 5 with 20/20 records from Producer-A
Sent batch 6 with 20/20 records from Producer-A
Sent batch 7 with 20/20 records from Producer-A
Sent batch 8 with 20/20 records from Producer-A
Sent batch 9 with 20/20 records from Producer-A
Sent batch 10 with 20/20 records from Producer-A
Sent batch 11 with 20/20 records from Producer-A
Sent batch 12 with 20/20 records from Producer-A
Sent batch 13 with 20/20 records from Producer-A
Sent batch 14 with 20/20 records from Producer-A
Sent batch 15 with 20/20 records from Producer-A
Sent batch 16 with 20/20 records from Producer-A
Sent batch 17 wi

Sent batch 163 with 20/20 records from Producer-A
Sent batch 164 with 20/20 records from Producer-A
Sent batch 165 with 20/20 records from Producer-A
Sent batch 166 with 20/20 records from Producer-A
Sent batch 167 with 20/20 records from Producer-A
Sent batch 168 with 20/20 records from Producer-A
Sent batch 169 with 20/20 records from Producer-A
Sent batch 170 with 20/20 records from Producer-A
Sent batch 171 with 20/20 records from Producer-A
Sent batch 172 with 20/20 records from Producer-A
Sent batch 173 with 20/20 records from Producer-A
Sent batch 174 with 20/20 records from Producer-A
Sent batch 175 with 20/20 records from Producer-A
Sent batch 176 with 20/20 records from Producer-A
Sent batch 177 with 20/20 records from Producer-A
Sent batch 178 with 20/20 records from Producer-A
Sent batch 179 with 20/20 records from Producer-A
Sent batch 180 with 20/20 records from Producer-A
Sent batch 181 with 20/20 records from Producer-A
Sent batch 182 with 20/20 records from Producer-A


/tmp/ipykernel_1922/2803335257.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  batch_df['event_date'] = pd.to_datetime(batch_df['timestamp']).dt.date.astype(str)


Sent batch 280 with 20/20 records from Producer-A
Sent batch 281 with 20/20 records from Producer-A
Sent batch 282 with 20/20 records from Producer-A
Sent batch 283 with 20/20 records from Producer-A
Sent batch 284 with 20/20 records from Producer-A
Sent batch 285 with 20/20 records from Producer-A
Sent batch 286 with 20/20 records from Producer-A
Sent batch 287 with 20/20 records from Producer-A
Sent batch 288 with 20/20 records from Producer-A
Sent batch 289 with 20/20 records from Producer-A
Sent batch 290 with 20/20 records from Producer-A
Sent batch 291 with 20/20 records from Producer-A
Sent batch 292 with 20/20 records from Producer-A
Sent batch 293 with 20/20 records from Producer-A
Sent batch 294 with 20/20 records from Producer-A
Sent batch 295 with 20/20 records from Producer-A
Sent batch 296 with 20/20 records from Producer-A
Sent batch 297 with 20/20 records from Producer-A
Sent batch 298 with 20/20 records from Producer-A
Sent batch 299 with 20/20 records from Producer-A


Sent batch 444 with 20/20 records from Producer-A
Sent batch 445 with 20/20 records from Producer-A
Sent batch 446 with 20/20 records from Producer-A
Sent batch 447 with 20/20 records from Producer-A
Sent batch 448 with 20/20 records from Producer-A
Sent batch 449 with 20/20 records from Producer-A
Sent batch 450 with 20/20 records from Producer-A
Sent batch 451 with 20/20 records from Producer-A
Sent batch 452 with 20/20 records from Producer-A
Sent batch 453 with 20/20 records from Producer-A
Sent batch 454 with 20/20 records from Producer-A
Sent batch 455 with 20/20 records from Producer-A
Sent batch 456 with 20/20 records from Producer-A
Sent batch 457 with 20/20 records from Producer-A
Sent batch 458 with 20/20 records from Producer-A
Sent batch 459 with 20/20 records from Producer-A
Sent batch 460 with 20/20 records from Producer-A
Sent batch 461 with 20/20 records from Producer-A
Sent batch 462 with 20/20 records from Producer-A
Sent batch 463 with 20/20 records from Producer-A
